<h3>准备</h3>

<h4>导入包</h4>

In [46]:
import os
import json
import requests
from time import localtime, strftime
from typing import Optional, Union
from PIL.Image import Image, open, fromarray
from PIL.ImageFilter import FIND_EDGES
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, LeakyReLU, Flatten
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import History, Callback, TensorBoard, EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.backend import clear_session
from sklearn.metrics import classification_report

<h4>设置tensorflow</h4>

In [2]:
for gpu in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

<h3>准备数据</h3>

In [3]:
train_dir = os.path.join("../data/pic/", "train")
test_dir = os.path.join("../data/pic/", "test")

<h4>边缘检测</h4>

In [4]:
def find_edges(img: Union[np.ndarray, Image]) -> np.ndarray:
    """
    图片边缘检测\n
    :param img: 一张图片
    :return: 边缘检测后的图片
    """
    if isinstance(img, np.ndarray):
        image: Image = fromarray(img.astype(np.uint8))
    else:
        image: Image = img
    return np.array(image.filter(FIND_EDGES)).astype("float")

<h4>数据预处理</h4>

In [19]:
train_gen = ImageDataGenerator(
    rescale=1.0 / 255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=360,
    width_shift_range=0.03,
    height_shift_range=0.03,
    fill_mode='nearest',
    preprocessing_function=lambda x: find_edges(x),
    validation_split=0.2
)
val_gen = ImageDataGenerator(
    rescale=1.0 / 255,
    preprocessing_function=lambda x: find_edges(x),
    validation_split=0.2
)

In [21]:
train_ds = train_gen.flow_from_directory(
    train_dir,
    target_size=(160, 160),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=123,
    subset="training"
)
val_ds = train_gen.flow_from_directory(
    train_dir,
    target_size=(160, 160),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=123,
    subset="validation"
)

Found 89 images belonging to 2 classes.
Found 21 images belonging to 2 classes.


<h3>搭建模型并训练</h3>

<h4>搭建模型</h4>

In [35]:
clear_session()
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), data_format="channels_last", activation="relu", name="conv2d1", input_shape=(160, 160, 3)),
    MaxPool2D(pool_size=(2, 2), name="maxpool2d_1"),
    Conv2D(filters=64, kernel_size=(3, 3), data_format="channels_last", activation="relu", name="conv2d2"),
    MaxPool2D(pool_size=(2, 2), name="maxpool2d_2"),
    Conv2D(filters=128, kernel_size=(3, 3), data_format="channels_last", activation="relu", name="conv2d3"),
    MaxPool2D(pool_size=(2, 2), name="maxpool2d_3"),
    Conv2D(filters=32, kernel_size=(3, 3), data_format="channels_last", activation="relu", name="conv2d4"),
    MaxPool2D(pool_size=(2, 2), name="maxpool2d_4"),
    Conv2D(filters=16, kernel_size=(3, 3), data_format="channels_last", activation="relu", name="conv2d5"),
    MaxPool2D(pool_size=(2, 2), name="maxpool2d_5"),
    Flatten(data_format="channels_last", name="flatten_6"),
    Dense(units=16, activation="relu", name="dense_6"),
    Dense(units=1, activation="sigmoid", name="dense_7")
], name="pic_clf")
model.compile(
    loss=BinaryCrossentropy(),
    optimizer=Adam(learning_rate=1e-4),
    metrics=["accuracy"]
)
model.summary()

Model: "pic_clf"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d1 (Conv2D)             (None, 158, 158, 32)      896       
_________________________________________________________________
maxpool2d_1 (MaxPooling2D)   (None, 79, 79, 32)        0         
_________________________________________________________________
conv2d2 (Conv2D)             (None, 77, 77, 64)        18496     
_________________________________________________________________
maxpool2d_2 (MaxPooling2D)   (None, 38, 38, 64)        0         
_________________________________________________________________
conv2d3 (Conv2D)             (None, 36, 36, 128)       73856     
_________________________________________________________________
maxpool2d_3 (MaxPooling2D)   (None, 18, 18, 128)       0         
_________________________________________________________________
conv2d4 (Conv2D)             (None, 16, 16, 32)        3689

<h4>模型训练</h4>

In [36]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=60,
    verbose=1,
    workers=-1,
    use_multiprocessing=True,
    callbacks=[
        TensorBoard(log_dir="../logs/" + strftime("%Y%m%d-%H%M%S", localtime()))
    ]
)

Epoch 1/60
3/3 [==============================] - 1s 333ms/step - loss: 0.6786 - accuracy: 0.6404 - val_loss: 0.6466 - val_accuracy: 0.7143
Epoch 2/60
3/3 [==============================] - 1s 200ms/step - loss: 0.6429 - accuracy: 0.7191 - val_loss: 0.6193 - val_accuracy: 0.7143
Epoch 3/60
3/3 [==============================] - 1s 201ms/step - loss: 0.6127 - accuracy: 0.7191 - val_loss: 0.6027 - val_accuracy: 0.7143
Epoch 4/60
3/3 [==============================] - 1s 203ms/step - loss: 0.5953 - accuracy: 0.7191 - val_loss: 0.5891 - val_accuracy: 0.7143
Epoch 5/60
3/3 [==============================] - 1s 217ms/step - loss: 0.5855 - accuracy: 0.7191 - val_loss: 0.5861 - val_accuracy: 0.7143
Epoch 6/60
3/3 [==============================] - 1s 233ms/step - loss: 0.5845 - accuracy: 0.7191 - val_loss: 0.5816 - val_accuracy: 0.7143
Epoch 7/60
3/3 [==============================] - 1s 214ms/step - loss: 0.5804 - accuracy: 0.7191 - val_loss: 0.5854 - val_accuracy: 0.7143
Epoch 8/60
3/3 [====

In [ ]:
tensorboard --logdir ../logs

In [40]:
save_model(model, "home/centos/pic_clf/1")      # save_model(model, "../models/pic_clf/1")

INFO:tensorflow:Assets written to: ../models/pic_clf/1\assets


<h3>部署模型并预测</h3>

<h4>部署模型</h4>

<h5>安装docker</h5>

In [ ]:
!sudo yum install -y epel-release
!sudo yum install https://get.docker.com/rpm/1.7.1/centos-6/RPMS/x86_64/docker-engine-1.7.1-1.el6.x86_64.rpm
!docker version
!sudo service docker start

<h5>拉取镜像</h5>

In [ ]:
!docker pull tensorflow/serving:2.3.0

<h5>部署服务</h5>

In [ ]:
!docker run -t \
    -p 8501:8501 \
    -v "/home/centos/pic_clf:/models/pic_clf" \
    -e MODEL_NAME=pic_clf \
    tensorflow/serving:2.3.0

<h4>测试数据</h4>

<h5>读取测试数据</h5>

In [50]:
x_test, y_test = list(image_dataset_from_directory(test_dir, image_size=(160, 160)))[1]

Found 63 files belonging to 2 classes.


In [51]:
x_test = x_test.numpy()
y_test = y_test.numpy()

<h5>给测试数据做预处理</h5>

In [52]:
for i in range(x_test.shape[0]):
    x_test[i, :, :, :] = find_edges(x_test[i]) / 255

<h5>预测数据</h5>

In [53]:
data = json.dumps({"instances": x_test.tolist()})
headers = {"content-type": "application/json"}
response = requests.post(
    url="http://81.70.8.71:8501/v1/models/pic_clf:predict",
    data=data,
    headers=headers
)

In [54]:
y_pred_prob = np.array(json.loads(response.text)["predictions"])

In [55]:
y_pred = np.where(y_pred_prob > 0.5, 1, 0).ravel()

In [56]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.86      0.92         7
           1       0.96      1.00      0.98        24

    accuracy                           0.97        31
   macro avg       0.98      0.93      0.95        31
weighted avg       0.97      0.97      0.97        31

